In [1]:
# Required imports
import os
import shutil
import time
import cv2
import numpy as np
import math
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision import datasets
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
import seaborn as sns

# Constants
Image.MAX_IMAGE_PIXELS = None
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
IMAGE_EXT = '.jpg'
BASE_PATH = '/workspace/0728tot/last_experiment/'
RESIZE_DIMS = (224, 224)
NUM_CLASSES = 2


!ln -s /workspace/0728tot/last_experiment/resnet.py resnet.py
from resnet import *

ln: failed to create symbolic link 'resnet.py': File exists


2023-10-25 17:35:34.507112: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
# Directory paths

# Folders containing original DUV WSI
ORIGINAL_IMAGES_DIR = os.path.join(BASE_PATH, 'unknown_DUV_WSI')

# Folder containing reshaped DUV WSI
RESHAPED_IMAGES_DIR = os.path.join(BASE_PATH, 'reshaped_unknown_DUV_WSI')

# Folders processing the GradCAM++ results
RAW_MAP_DIR = os.path.join(BASE_PATH, 'unknown_DenseNet169ImageNetGradCAM_2')
FINAL_HEAT_MAP_DIR = os.path.join(BASE_PATH, 'unknown_DenseNet169ImageNetGradCAM')
FINAL_OVERLAY_HEAT_MAP_DIR = os.path.join(BASE_PATH, 'unknown_DUV_finalheatmap/')

# Folders for holding the GradCAM++ results and patch level results in a grid system
GRADCAM_GRID_MAP_DIR = os.path.join(BASE_PATH, 'unknown_gradcam_gridmap')
PRED_GRID_MAP_DIR = os.path.join(BASE_PATH, 'unknown_patch_gridmap')

# Folders for processing the patch level and GradCAM++ results for visualization purposes
PROCESS_PATCHES_DIR = os.path.join(BASE_PATH,'unknown_process_patchmaps')
BOUNDING_BOXES_OVERLAY_DIR = os.path.join(BASE_PATH,'unknown_bounding_boxes_overlays')

# Folders for patch related info
PATCH_DIR = os.path.join(BASE_PATH, 'unknown_Patches')
PATCH_FEATURES_DIR = os.path.join(BASE_PATH, 'unknown_Patches_features')
PRIOR_PATCH_FEATURES_DIR = os.path.join(BASE_PATH, 'DUV_features')
PRIOR_PATCH_DATA_CSV = os.path.join(BASE_PATH, 'duvdata.csv')

# Folder for getting training splits, an example
SPLIT_DIR = os.path.join(BASE_PATH, 'prior_train_splits')

In [3]:
# Load Data
def make_old_dataframe(bh_folder, ext, name_replace_list, column_mapping):
    '''Utility function to create dataframe from dataset'''    
    data_entries = []
    for root, _, files in os.walk(bh_folder):
        for filename in files:
            relative_path = os.path.relpath(root, bh_folder)
            file_extension = os.path.splitext(filename)[-1]
            if file_extension == ext:
                try:
                    full_path = os.path.join(root, filename)  # Compute the full path using the original filename
                    for old, new in name_replace_list:
                        filename = filename.replace(old, new)
                    file_data = filename.replace(ext, "").split('_')
                    row_data = dict(zip(column_mapping, [relative_path, filename, full_path] + file_data))
                    data_entries.append(row_data)
                except:
                    pass
    df = pd.DataFrame(data_entries)
    for col, dtype in zip(column_mapping[3:], ['int'] * len(column_mapping[3:])):
        df[col] = df[col].astype(dtype)
    return df

# Create DataFrames
column_mapping_patch = ['subject', 'filename', 'fullpath', 'seq', 'y', 'x']

duv_patch_df = make_old_dataframe(PATCH_DIR, '.png', [("-", "_")], column_mapping_patch)

In [4]:
# # Feature Extraction
# You will have to restart kernel you don't have enough GPU VRAM, check with !nvidia-smi

if 'RNObject' not in locals():
    print('Making ResNet object.')
    RNObject = ResNetClass()

res50 = RNObject.get_resnet_model(layer_name='avg_pool')
RNObject.save_resnet_predictions(duv_patch_df, DIF_PATCH_FEATURES_DIR, res50, verbose=1, mode=1)